<a href="https://colab.research.google.com/github/tvelichkovt/TensorFlow/blob/master/5_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print('TensorFlow ver is:', tf.__version__, '\nKeras ver is:', keras.__version__, '\nPandas ver is:', pd.__version__, '\nNumpy ver is:', np.__version__)

TensorFlow ver is: 2.1.0 
Keras ver is: 2.2.4-tf 
Pandas ver is: 0.25.3 
Numpy ver is: 1.17.5


**Load** and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

---



In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

tf.keras.Sequential

---



In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [0]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.09812807, -0.27887037,  0.11286126, -0.25341052,  0.3577938 ,
         0.27823484,  0.53416675,  0.5049786 , -0.07065955,  0.31877032]],
      dtype=float32)

tf.nn.softmax

---



In [0]:
tf.nn.softmax(predictions).numpy()

array([[0.09059267, 0.06213906, 0.09193727, 0.06374143, 0.11745309,
        0.10847069, 0.14010778, 0.1360774 , 0.07652257, 0.11295795]],
      dtype=float32)

losses.SparseCategoricalCrossentropy

---



In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.log(1/10) ~= 2.3.

---



In [0]:
loss_fn(y_train[:1], predictions).numpy()

2.2212753

In [0]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

model.fit

---



In [0]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 2.2932 - accuracy: 0.1183


[2.2932360885620118, 0.1183]

The image classifier is now trained to ~98% accuracy on this dataset. If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

---



In [0]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [0]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.12069602, 0.08754577, 0.13055806, 0.05593909, 0.13371553,
        0.06033571, 0.11470913, 0.10544477, 0.08681696, 0.10423902],
       [0.12058563, 0.08995616, 0.07891829, 0.06785422, 0.14394471,
        0.11633885, 0.1047703 , 0.07147906, 0.08769495, 0.11845797],
       [0.10687049, 0.11032632, 0.105048  , 0.08387838, 0.12890634,
        0.07602976, 0.10212784, 0.11916203, 0.09166948, 0.07598146],
       [0.13684995, 0.11521091, 0.07163788, 0.04377413, 0.19715711,
        0.06985612, 0.15044259, 0.06427581, 0.08283724, 0.06795822],
       [0.13377872, 0.08282432, 0.14003213, 0.05005923, 0.13251041,
        0.06191589, 0.13233101, 0.06347001, 0.12811042, 0.07496782]],
      dtype=float32)>